In [1]:
!pip install -q -U accelerate transformers peft pip install protobuf sentencepiece

In [2]:
!git config --global credential.helper store

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
# Cell 1: Imports
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

In [6]:
# Cell 2: Parameters (replace the values with your actual paths and preferences)
base_model_name_or_path = "codellama/CodeLlama-34b-Python-hf"
peft_model_path = "TokenBender/UnnaturalCodeLLama-P-adapter"
output_dir = "UnnaturalCodellama_P"
device = "cuda:0"  # or specify a specific device like "cuda:0"
push_to_hub = False  # or True if you want to push to the hub

In [4]:
# Cell 3: Load Base Model
print(f"Loading base model: {base_model_name_or_path}")
if device == 'auto':
    device_arg = { 'device_map': 'auto' }
else:
    device_arg = { 'device_map': { "": device} }

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    return_dict=True,
    torch_dtype=torch.float16,
    **device_arg
)

Loading base model: codellama/CodeLlama-34b-Python-hf


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
# Cell 4: Load PEFT and Merge
print(f"Loading PEFT: {peft_model_path}")
model = PeftModel.from_pretrained(base_model, peft_model_path, **device_arg)
print(f"Running merge_and_unload")
model = model.merge_and_unload()

Loading PEFT: TokenBender/UnnaturalCodeLLama-P-adapter
Running merge_and_unload


In [7]:
# Cell 5: Tokenizer and Saving
tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path)

if push_to_hub:
    print(f"Saving to hub ...")
    model.push_to_hub(f"{output_dir}", use_temp_dir=True)
    tokenizer.push_to_hub(f"{output_dir}", use_temp_dir=True)
else:
    model.save_pretrained(f"{output_dir}")
    tokenizer.save_pretrained(f"{output_dir}")
    print(f"Model saved to {output_dir}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model saved to UnnaturalCodellama_P


## When push_to_hub is set to False and we push manually separately

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(output_dir, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

model.push_to_hub(f"{output_dir}")
tokenizer.push_to_hub(f"{output_dir}")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

pytorch_model-00002-of-00007.bin:   0%|          | 0.00/9.69G [00:00<?, ?B/s]

pytorch_model-00004-of-00007.bin:   0%|          | 0.00/9.69G [00:00<?, ?B/s]

pytorch_model-00003-of-00007.bin:   0%|          | 0.00/9.69G [00:00<?, ?B/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/9.85G [00:00<?, ?B/s]

pytorch_model-00005-of-00007.bin:   0%|          | 0.00/9.69G [00:00<?, ?B/s]

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/9.69G [00:00<?, ?B/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/9.19G [00:00<?, ?B/s]

In [ ]:
!pip install numba -q -U

In [10]:
from numba import cuda
device = cuda.get_current_device()
device.reset()